In [10]:
import os
import re
import pandas as pd

import nltk
nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import string

from collections import Counter



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cynhe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
data = pd.read_csv('./final_tracking.csv')
data.columns.to_list()

['Unnamed: 0', 'Lab', 'Study explanation', 'Study title', 'Phase']

In [12]:
#data_p = data.rename(columns={'General Subject Matter':'general_subject'})

In [17]:
data.columns.to_list()

['Unnamed: 0', 'Lab', 'Study explanation', 'Study title', 'Phase']

In [18]:
def tokenizer(text):
    stop_words = set(stopwords.words('english'))
    punct = string.punctuation+"¿?¡"
    tokens = word_tokenize(text)
#    palabraLower = 
    filtered_tokens = [word.lower() for word in tokens if not word.lower() in stop_words and word.lower() not in punct]
    
    #Roberto dice que este word.lower no está my bien porque se repite mucho, es mejor hacer una variable anterior para anidarla
    return filtered_tokens

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         88 non-null     int64 
 1   Lab                88 non-null     object
 2   Study explanation  88 non-null     object
 3   Study title        50 non-null     object
 4   Phase              50 non-null     object
dtypes: int64(1), object(4)
memory usage: 3.6+ KB


In [21]:
data['tokenized_text'] = data['Study explanation'].apply(tokenizer)

In [ ]:
#data_p['tokens'] = data_p['general_subject'].apply(lambda x: nltk.word_tokenize(x))

In [23]:
data

,Unnamed: 0,Lab,Study explanation,Study title,Phase,tokenized_text
0,0,MAPS PBC,MAPS has completed conducting two Phase 3 tria...,Post-traumatic stress disorder (PTSD) | MDMA,Phase 3,"[maps, completed, conducting, two, phase, 3, t..."
1,1,Compass Pathways,COMP360 is a psilocybin formulation currently ...,Treatment-resistant depression (TRD) | COMP3...,Phase 3,"[comp360, psilocybin, formulation, currently, ..."
2,2,Awakn Life Sciences,Awakn is set to evaluate ketamine-assisted psy...,Alcohol use disorder (AUD) | Ketamine,Phase 3,"[awakn, set, evaluate, ketamine-assisted, psyc..."
3,3,GH Research,"GH001 is an inhaled formulation of 5-MeO-DMT, ...",Treatment-resistant depression (TRD)| GH001 (5...,Phase 2b,"[gh001, inhaled, formulation, 5-meo-dmt, short..."
4,4,MindMed,MM-120 is a formulation of LSD being evaluated...,Generalized Anxiety Disorder (GAD) | MM-120 (...,Phase 2b,"[mm-120, formulation, lsd, evaluated, phase, 2..."
...,...,...,...,...,...,...
83,83,PSYLO,Psylo is currently in the discovery stages of ...,NaN,NaN,"[psylo, currently, discovery, stages, developm..."
84,84,Mindstate Design,"MSD-101, MSD-102, MSD-103, MSD-104, MSD-105 Di...",NaN,NaN,"[msd-101, msd-102, msd-103, msd-104, msd-105, ..."
85,85,Lusaris Therapeutics,Lusaris is developing a portfolio of new compo...,NaN,NaN,"[lusaris, developing, portfolio, new, compound..."
86,86,Beckley Psytech,NCE discovery program.,NaN,NaN,"[nce, discovery, program]"


In [24]:
palabras = [word for sentence in data.tokenized_text.to_list() for word in sentence]

r = Counter(palabras)
print ("Palabras:  ", r.most_common(35))

Palabras:   [('trial', 79), ('phase', 63), ('potential', 42), ('study', 41), ('evaluated', 41), ('treatment', 37), ('currently', 34), ('clinical', 34), ('details', 34), ('found', 33), ('disorder', 32), ('registration', 32), ('link', 31), ('use', 30), ('formulation', 29), ('preclinical', 26), ('development', 21), ('2', 20), ('psilocybin', 18), ('depression', 18), ('unknown', 14), ('compounds', 14), ('stage', 14), ('1', 12), ('treatment-resistant', 11), ('set', 11), ('drug', 11), ('therapy', 10), ('major', 10), ('depressive', 10), ('intended', 10), ('including', 10), ('synthetic', 9), ('gh001', 8), ('5-meo-dmt', 8)]


INICIA NLTK

In [25]:
import gensim

In [26]:
tokens_solos = data['tokenized_text'].tolist()
dictionary = gensim.corpora.Dictionary(tokens_solos)

In [27]:
dictionary

In [28]:
dictionary.filter_extremes(no_below=4, no_above=0.8, keep_n=100000)

In [29]:
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokens_solos]

In [30]:
LDA = gensim.models.ldamodel.LdaModel
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, random_state=100, chunksize=1000, passes=50, iterations=100)

In [31]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [32]:
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

c:\Users\cynhe\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.222398  0.008634       1        1  57.001587
4     -0.080092  0.038503       2        1  12.433128
0      0.177510 -0.070617       3        1  12.076201
2      0.011145 -0.144770       4        1  10.858785
1      0.113835  0.168249       5        1   7.630298, topic_info=           Term       Freq      Total Category  logprob  loglift
28          use  28.000000  28.000000  Default  30.0000  30.0000
18  formulation  27.000000  27.000000  Default  29.0000  29.0000
17    evaluated  39.000000  39.000000  Default  28.0000  28.0000
46  development  19.000000  19.000000  Default  27.0000  27.0000
53  preclinical  24.000000  24.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
58       opioid   0.184473   5.612942   Topic5  -6.1338  -0.8423
46  development   0.191791  19.883459   Topic5  -6.0949  -2.0682
3      disorder   0.189985  31.000600   Topic5  -6.1044  -2.5218
17    evaluated   0.186460  39.226612   Topic5  -6.1231  -2.7759
18  formulation   0.184360  27.922736   Topic5  -6.1344  -2.4473

[213 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
55        1  0.414987            1
55        2  0.497985            1
55        3  0.082997            1
29        1  0.959223            2
69        4  0.865069  2-bromo-lsd
...     ...       ...          ...
67        5  0.585203      unknown
28        1  0.622132          use
28        2  0.034563          use
28        4  0.034563          use
28        5  0.276503          use

[182 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 1, 3, 2])